In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import pickle
import datetime
from datetime import datetime
from pytz import timezone
import pytz
from pytz import all_timezones
import pickle

In [2]:
path="C:\\Users\\11147\\Desktop\\Sanket_backup\\Mhack\\"

In [3]:
df=pd.read_parquet(r"C:\Users\11147\Desktop\Sanket_backup\Mhack\drive\drive_1557398801351_trip_1557398302922_part-00000-tid-4787120893022696848-4897608f-5b93-42b5-a8ab-e90f5cadf0bf-1853-c000.snappy.parquet",engine="pyarrow")

In [4]:
drive_col=df.columns.tolist()
drive_col

['vehicle_id',
 'trip_id',
 'datetime',
 'velocity',
 'accel_x',
 'accel_y',
 'accel_z',
 'engine_coolant_temp',
 'eng_load',
 'fuel_level',
 'iat',
 'rpm']

In [5]:
df_drive=pd.DataFrame(columns=drive_col)

In [6]:
for j in os.listdir(path + "drive"):
    df=pd.read_parquet(path+"drive"+ "\\" +str(j),engine="pyarrow")
    df_drive=pd.concat([df_drive,df],ignore_index=True)

In [7]:
df_drive.shape

(9217528, 12)

In [8]:
df_drive.head()

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm
0,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:00,0.00,50.0,65.13,87.92,150.70,192.88,118.66,95.0,2107.44
1,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:01,47.49,51.0,72.34,83.78,154.76,196.04,116.69,100.0,2119.17
2,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:02,57.94,50.0,69.20,87.73,165.50,196.01,105.46,90.0,2112.32
3,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:03,57.50,48.0,70.83,84.95,157.32,193.31,110.35,90.0,2092.94
4,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:04,52.29,47.0,70.22,92.90,151.00,189.49,104.25,97.0,2117.87


In [11]:
target_df=pd.DataFrame(columns=["trip_id","ft_cnt_vehicle_deaccel_val","ft_cnt_vehicle_accel_val"])

In [12]:
total_trip_id=df_drive.trip_id.unique().tolist()

In [11]:
for trip in total_trip_id:
    tr=df_drive[df_drive.trip_id==trip]
    tr_sub=tr.velocity.tolist()
    variation=[]
    for i in range(0,len(tr_sub)-1):
        if tr_sub[i]>tr_sub[i+1]:
            variation.append("d")
        elif tr_sub[i]<tr_sub[i+1]:
            variation.append("a")
    if len(variation)==0:
        deacc_val=0
        acc_val=0
    else:
        change=[]
        for i in range(0,len(variation)-1):
            if variation[i]==variation[i+1]:
                pass
            else:
                change.append(variation[i])
        if variation[-1]==variation[-2]:
            pass
        else:
            change.append(variation[-1])
        deacc_val=change.count("d")
        acc_val=change.count("a")
    target_df=target_df.append({"trip_id":trip,"ft_cnt_vehicle_deaccel_val":change.count("d"),"ft_cnt_vehicle_accel_val":change.count("a")},ignore_index=True)

In [25]:
pickle_in = open("data.pickle","rb")
target_df = pickle.load(pickle_in)

In [26]:
target_df.head()

,trip_id,ft_cnt_vehicle_deaccel_val,ft_cnt_vehicle_accel_val,ft_sum_hard_brakes_10_flg_val,ft_sum_hard_brakes_3_flg_val,ft_sum_hard_accel_10_flg_val,ft_sum_hard_accel_10_flg_val,ft_sum_time_deaccel_val,dt_sumtime_accel_val
0,db3c08b01205459294bbc91e3f7dfeb6,1429,1430,NaN,NaN,NaN,NaN,NaN,NaN
1,77b52511437f4b78871eb1ef6080daae,2381,2382,NaN,NaN,NaN,NaN,NaN,NaN
2,b086a49b14b4420c98bbb569510c5629,894,895,NaN,NaN,NaN,NaN,NaN,NaN
3,726358cae8754f52a9fcdd0d1441d618,213,214,NaN,NaN,NaN,NaN,NaN,NaN
4,1db28627c3c748868f4eb7c6afb8d083,1334,1335,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#Hard accel or deaacel

In [ ]:
speed_var=[]
for i in range(0,len(a_sub)-1):
    speed_var.append(a_sub[i+1]-a_sub[i])

In [19]:
deacc=[]

In [20]:
for trip in total_trip_id:
    speed_var=[]
    tr=df_drive[df_drive.trip_id==trip]
    tr_sub=tr.velocity.tolist()
    for speed in range(0,len(tr_sub)-1):
        speed_var.append(tr_sub[speed+1]-tr_sub[speed])
    if len(speed_var)==1:
        dec_10=0
    else:
        new_speed_deaccel=[x if x<=-10 else "no" for x in speed_var]
        hard_deaccel=[]
        for i in range(0,len(new_speed_deaccel)-1):
            if new_speed_deaccel[i]==new_speed_deaccel[i+1]:
                pass
            else:
                hard_deaccel.append(new_speed_deaccel[i])
        if new_speed_deaccel[-1]==new_speed_deaccel[-2]:
            pass
        else:
            hard_deaccel.append(new_speed_deaccel[-1])

        hard_deaccel.append("no")
        hard_deaccel.insert(0,"no")
        
        m=[]
        z=0
        for i in range(0,len(hard_deaccel)-1):
            if type(hard_deaccel[i])!=type(hard_deaccel[i+1]):
                if type(hard_deaccel[i])==int and type(hard_deaccel[i-1])==int:
                    pass
                else:
                    m.append(hard_deaccel[i])
                z=0
            elif type(hard_deaccel[i])==type(hard_deaccel[i+1])==int:
                v=[z,hard_deaccel[i],hard_deaccel[i+1]]
                z=max(v)
                if hard_deaccel[i+2]=="n":
                    m.append(z)
            else:
                pass
            
            dec_10=sum(isinstance(x, float) for x in m)
    deacc.append(dec_10)

In [28]:
target_df["ft_sum_hard_brakes_10_flg_val"]=deacc

In [42]:
deacc_3_10=[]

In [43]:
for trip in total_trip_id:
    speed_var=[]
    tr=df_drive[df_drive.trip_id==trip]
    tr_sub=tr.velocity.tolist()
    for speed in range(0,len(tr_sub)-1):
        speed_var.append(tr_sub[speed+1]-tr_sub[speed])
    if len(speed_var)==1:
        dec_10=0
    else:
        new_speed_deaccel=[x if -3>= x >-10 else "no" for x in speed_var]
        hard_deaccel=[]
        for i in range(0,len(new_speed_deaccel)-1):
            if new_speed_deaccel[i]==new_speed_deaccel[i+1]:
                pass
            else:
                hard_deaccel.append(new_speed_deaccel[i])
        if new_speed_deaccel[-1]==new_speed_deaccel[-2]:
            pass
        else:
            hard_deaccel.append(new_speed_deaccel[-1])

        hard_deaccel.append("no")
        hard_deaccel.insert(0,"no")
        
        m=[]
        z=0
        for i in range(0,len(hard_deaccel)-1):
            if type(hard_deaccel[i])!=type(hard_deaccel[i+1]):
                if type(hard_deaccel[i])==int and type(hard_deaccel[i-1])==int:
                    pass
                else:
                    m.append(hard_deaccel[i])
                z=0
            elif type(hard_deaccel[i])==type(hard_deaccel[i+1])==int:
                v=[z,hard_deaccel[i],hard_deaccel[i+1]]
                z=max(v)
                if hard_deaccel[i+2]=="n":
                    m.append(z)
            else:
                pass
            
            dec_10=sum(isinstance(x, float) for x in m)
    deacc_3_10.append(dec_10)

In [44]:
target_df["ft_sum_hard_brakes_3_flg_val"]=deacc_3_10

In [30]:
acc=[]

In [31]:
for trip in total_trip_id:
    speed_var=[]
    tr=df_drive[df_drive.trip_id==trip]
    tr_sub=tr.velocity.tolist()
    for speed in range(0,len(tr_sub)-1):
        speed_var.append(tr_sub[speed+1]-tr_sub[speed])
    if len(speed_var)==1:
        acc_10=0
    else:
        new_speed_accel=[x if x > 10 else "no" for x in speed_var]
        hard_accel=[]
        for i in range(0,len(new_speed_accel)-1):
            if new_speed_accel[i]==new_speed_accel[i+1]:
                pass
            else:
                hard_accel.append(new_speed_accel[i])
        if new_speed_accel[-1]==new_speed_accel[-2]:
            pass
        else:
            hard_accel.append(new_speed_accel[-1])

        hard_accel.append("no")
        hard_accel.insert(0,"no")
        
        m=[]
        z=0
        for i in range(0,len(hard_accel)-1):
            if type(hard_accel[i])!=type(hard_accel[i+1]):
                if type(hard_accel[i])==int and type(hard_accel[i-1])==int:
                    pass
                else:
                    m.append(hard_accel[i])
                z=0
            elif type(hard_accel[i])==type(hard_accel[i+1])==int:
                v=[z,hard_accel[i],hard_accel[i+1]]
                z=max(v)
                if hard_accel[i+2]=="n":
                    m.append(z)
            else:
                pass
            
            acc_10=sum(isinstance(x, float) for x in m)
    acc.append(acc_10)

In [52]:
target_df["ft_sum_hard_accel_10_flg_val"]=acc

In [56]:
acc_3_10=[]

In [57]:
for trip in total_trip_id:
    speed_var=[]
    tr=df_drive[df_drive.trip_id==trip]
    tr_sub=tr.velocity.tolist()
    for speed in range(0,len(tr_sub)-1):
        speed_var.append(tr_sub[speed+1]-tr_sub[speed])
    if len(speed_var)==1:
        acc_10=0
    else:
        new_speed_accel=[x if 3<= x < 10 else "no" for x in speed_var]
        hard_accel=[]
        for i in range(0,len(new_speed_accel)-1):
            if new_speed_accel[i]==new_speed_accel[i+1]:
                pass
            else:
                hard_accel.append(new_speed_accel[i])
        if new_speed_accel[-1]==new_speed_accel[-2]:
            pass
        else:
            hard_accel.append(new_speed_accel[-1])

        hard_accel.append("no")
        hard_accel.insert(0,"no")
        
        m=[]
        z=0
        for i in range(0,len(hard_accel)-1):
            if type(hard_accel[i])!=type(hard_accel[i+1]):
                if type(hard_accel[i])==int and type(hard_accel[i-1])==int:
                    pass
                else:
                    m.append(hard_accel[i])
                z=0
            elif type(hard_accel[i])==type(hard_accel[i+1])==int:
                v=[z,hard_accel[i],hard_accel[i+1]]
                z=max(v)
                if hard_accel[i+2]=="n":
                    m.append(z)
            else:
                pass
            
            acc_10=sum(isinstance(x, float) for x in m)
    acc_3_10.append(acc_10)

In [58]:
target_df["ft_sum_hard_accel_3_flg_val"]=acc_3_10

In [59]:
target_df.head(10)

,trip_id,ft_cnt_vehicle_deaccel_val,ft_cnt_vehicle_accel_val,ft_sum_hard_brakes_10_flg_val,ft_sum_hard_brakes_3_flg_val,ft_sum_hard_accel_10_flg_val,ft_sum_hard_accel_3_flg_val
0,db3c08b01205459294bbc91e3f7dfeb6,1429,1430,348,908,356,912
1,77b52511437f4b78871eb1ef6080daae,2381,2382,1948,824,1929,810
2,b086a49b14b4420c98bbb569510c5629,894,895,642,396,644,369
3,726358cae8754f52a9fcdd0d1441d618,213,214,160,90,166,85
4,1db28627c3c748868f4eb7c6afb8d083,1334,1335,1099,450,1085,459
5,808c3cd2a81b48eb8899213d7a106321,1239,1240,575,669,562,687
6,c398182d90c34132aad863caea0ce1bc,474,475,225,267,228,262
7,82be6387aaf845f28fd7ef10230fba9b,622,623,467,237,476,225
8,fd90bffab2bd460eaeb14de54e0a61a6,1429,1430,1053,579,1064,555
9,a73a1ad0cade477a99f95f694b792107,927,928,714,369,723,322


In [72]:
trip_acl=[]
trip_dcl=[]

In [73]:
for trip in total_trip_id:
    acl=0
    decl=0
    tr=df_drive[df_drive.trip_id==trip]
    tr_sub=tr.velocity.tolist()
    if sum(tr_sub)==0:
        acl+=0
        decl+=0
    else:
        for i in range(0,len(tr_sub)-1):
            if tr_sub[i]<tr_sub[i+1]:
                acl+=1
            else:
                decl+=1
    trip_acl.append(acl)
    trip_dcl.append(decl)
    

In [75]:
target_df["ft_sum_time_deaccel_val"]=trip_dcl
target_df["ft_sum_time_accel_val"]=trip_acl

In [74]:
len(trip_acl),len(trip_dcl)

(1708, 1708)

In [76]:
target_df.head(10)

,trip_id,ft_cnt_vehicle_deaccel_val,ft_cnt_vehicle_accel_val,ft_sum_hard_brakes_10_flg_val,ft_sum_hard_brakes_3_flg_val,ft_sum_hard_accel_10_flg_val,ft_sum_hard_accel_3_flg_val,ft_sum_time_deaccel_val,ft_sum_time_accel_val
0,db3c08b01205459294bbc91e3f7dfeb6,1429,1430,348,908,356,912,2142,2107
1,77b52511437f4b78871eb1ef6080daae,2381,2382,1948,824,1929,810,3563,3563
2,b086a49b14b4420c98bbb569510c5629,894,895,642,396,644,369,1385,1319
3,726358cae8754f52a9fcdd0d1441d618,213,214,160,90,166,85,324,320
4,1db28627c3c748868f4eb7c6afb8d083,1334,1335,1099,450,1085,459,2035,2009
5,808c3cd2a81b48eb8899213d7a106321,1239,1240,575,669,562,687,1812,1843
6,c398182d90c34132aad863caea0ce1bc,474,475,225,267,228,262,712,726
7,82be6387aaf845f28fd7ef10230fba9b,622,623,467,237,476,225,911,906
8,fd90bffab2bd460eaeb14de54e0a61a6,1429,1430,1053,579,1064,555,2152,2085
9,a73a1ad0cade477a99f95f694b792107,927,928,714,369,723,322,1421,1391


In [77]:
pickle_out = open("data.pickle","wb")
pickle.dump(target_df, pickle_out)
pickle_out.close()

In [78]:
pickle_in = open("data.pickle","rb")
target_df = pickle.load(pickle_in)

In [80]:
target_df.head()

,trip_id,ft_cnt_vehicle_deaccel_val,ft_cnt_vehicle_accel_val,ft_sum_hard_brakes_10_flg_val,ft_sum_hard_brakes_3_flg_val,ft_sum_hard_accel_10_flg_val,ft_sum_hard_accel_3_flg_val,ft_sum_time_deaccel_val,ft_sum_time_accel_val
0,db3c08b01205459294bbc91e3f7dfeb6,1429,1430,348,908,356,912,2142,2107
1,77b52511437f4b78871eb1ef6080daae,2381,2382,1948,824,1929,810,3563,3563
2,b086a49b14b4420c98bbb569510c5629,894,895,642,396,644,369,1385,1319
3,726358cae8754f52a9fcdd0d1441d618,213,214,160,90,166,85,324,320
4,1db28627c3c748868f4eb7c6afb8d083,1334,1335,1099,450,1085,459,2035,2009


In [81]:
target_df.to_csv("drive_features.csv",index=False)

In [9]:
################################################################################################################################

Engine csv

In [11]:
df_drive.head()

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm
0,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:00,0.00,50.0,65.13,87.92,150.70,192.88,118.66,95.0,2107.44
1,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:01,47.49,51.0,72.34,83.78,154.76,196.04,116.69,100.0,2119.17
2,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:02,57.94,50.0,69.20,87.73,165.50,196.01,105.46,90.0,2112.32
3,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:03,57.50,48.0,70.83,84.95,157.32,193.31,110.35,90.0,2092.94
4,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:04,52.29,47.0,70.22,92.90,151.00,189.49,104.25,97.0,2117.87


In [12]:
df_vehicle=pd.read_csv(r"C:\Users\11147\Desktop\Sanket_backup\Mhack\vehicle.csv")

In [14]:
df_vehicle

,vehicle_id,year,make,Model,drivetrain,max_torque,max_horsepower,max_horsepower_rpm,max_torque_rpm,engine_displacement,fuel_type,fuel_tank_capacity,fuel_economy_city,fuel_economy_highway,cylinders,forced_induction,device_generation
0,1000500,2016,Honda,Civic,2,174,140,6500,1500,1.799,1059,47,16.50,20.00,4,1054.0,5
1,1000501,2016,Jeep,Compass,4,350,171,3750,1250,1.956,1059,60,17.10,21.20,3,NaN,4
2,1000502,2016,Hyundai,Creta,2,260,126,4000,1500,1.582,1059,55,19.67,24.10,4,1054.0,3
3,1000503,2016,Skoda,Superb,2,250,177,5100,1750,1.798,1059,66,14.67,23.30,3,NaN,2
4,1000504,2017,Volkswagen,Passat,4,350,174,3600,1500,1.968,1059,66,17.42,20.43,4,NaN,4
5,1000506,2017,BMW,3 Series GT,2,400,188,4000,1800,1.995,1059,57,21.76,26.32,3,NaN,3
6,1000507,2017,Audi,Q3,3,250,148,5000,1400,1.395,1059,64,16.90,21.40,3,1054.0,2


In [16]:
vehicle=df_drive.vehicle_id.unique().tolist()

In [46]:
engine=df_drive[["vehicle_id","eng_load","rpm","datetime"]]

In [48]:
engine.sort_values("vehicle_id",inplace=True)    

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [49]:
engine["torque_utilization"]=engine["eng_load"]/255

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [169]:
engine["max_torque"]=0
engine["max_hp"]=0
engine["max_hp_rpm"]=0

In [170]:
for veh in vehicle:
    if veh in (df_vehicle.vehicle_id.tolist()):
        m_t=df_vehicle[df_vehicle.vehicle_id==veh]["max_torque"].values
        m_h=df_vehicle[df_vehicle.vehicle_id==veh]["max_horsepower"].values
        m_h_r=df_vehicle[df_vehicle.vehicle_id==veh]["max_horsepower_rpm"].values
        engine.loc[engine.vehicle_id==veh,"max_torque"]=m_t[0]
        engine.loc[engine.vehicle_id==veh,"max_hp"]=m_h[0]
        engine.loc[engine.vehicle_id==veh,"max_hp_rpm"]=m_h_r[0]
    else:
        pass

In [174]:
engine.drop(["hp_uti","tor_uti","rpm_uti"],axis=1,inplace=True)

In [176]:
engine.drop("active_hp",axis=1,inplace=True)

In [178]:
engine.drop("torque_utilization",axis=1,inplace=True)

In [181]:
engine_with=engine.loc[engine["vehicle_id"].isin(df_vehicle.vehicle_id.tolist())]

In [184]:
engine_without=engine.loc[~engine["vehicle_id"].isin(df_vehicle.vehicle_id.tolist())]

In [186]:
engine_without.shape

(5927875, 7)

In [188]:
engine_with.shape

(3289653, 7)

In [190]:
engine_with["active_hp"]=engine_with["eng_load"]/255 * engine_with["max_torque"] * engine_with["rpm"] / 5252

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [192]:
engine_with["hp_utl"]=engine_with["active_hp"] / engine_with["max_hp"]

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [197]:
engine_with["tor_utl"]=engine_with["eng_load"]/255

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [199]:
engine_with["rpm_uti"]=engine_with["rpm"]/engine_with["max_hp_rpm"]

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [204]:
engine_without["active_hp"]=0

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [206]:
engine_without["hp_utl"]=0

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [208]:
engine_without["tor_utl"]=engine_without["eng_load"]/255

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [210]:
engine_without["rpm_uti"]=0

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [211]:
engine_with.shape

(3289653, 11)

In [212]:
engine_without.shape

(5927875, 11)

In [213]:
new_engine=pd.concat([engine_with,engine_without],ignore_index=True)

In [215]:
new_engine

,vehicle_id,eng_load,rpm,datetime,max_torque,max_hp,max_hp_rpm,active_hp,hp_utl,tor_utl,rpm_uti
0,1000500,194.21,2105.21,2017-01-06 21:09:53,174,140,6500,53.119180,0.379423,0.761608,0.323878
1,1000500,199.97,1976.16,2017-01-06 22:22:33,174,140,6500,51.341825,0.366727,0.784196,0.304025
2,1000500,205.29,1974.85,2017-01-06 22:22:32,174,140,6500,52.672782,0.376234,0.805059,0.303823
3,1000500,193.45,1976.65,2017-01-06 22:22:31,174,140,6500,49.680146,0.354858,0.758627,0.304100
4,1000500,210.08,1976.78,2017-01-06 22:22:30,174,140,6500,53.954466,0.385389,0.823843,0.304120
5,1000500,202.76,1976.89,2017-01-06 22:22:29,174,140,6500,52.077381,0.371981,0.795137,0.304137
6,1000500,196.39,1976.11,2017-01-06 22:22:28,174,140,6500,50.421393,0.360153,0.770157,0.304017
7,1000500,201.24,1984.85,2017-01-06 22:22:27,174,140,6500,51.895100,0.370679,0.789176,0.305362
8,1000500,204.22,1974.36,2017-01-06 22:22:26,174,140,6500,52.385243,0.374180,0.800863,0.303748
9,1000500,205.85,1976.72,2017-01-06 22:22:25,174,140,6500,52.866478,0.377618,0.807255,0.304111


In [216]:
pickle_out = open("engine.pickle","wb")
pickle.dump(new_engine, pickle_out)
pickle_out.close()

In [217]:
pickle_in = open("engine.pickle","rb")
new_engine = pickle.load(pickle_in)